In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import glob
import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
import torch.optim as optim
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
import matplotlib.pyplot as plt

# DATA CLEANING

In [ ]:
# --- CONFIG ---
BASE_DIR = "/MALLORN-Astronomical-Classification-Challenge/data/raw"

# Hệ số bước sóng cho các filter (dùng để khử Extinction)
LAMBDA = {
    'u': 4.81,
    'g': 3.64,
    'r': 2.70,
    'i': 2.06,
    'z': 1.58,
    'y': 1.30
}

def process_data(mode='train'):
    print(f"--- Processing {mode} data ---")
    log_file = os.path.join(BASE_DIR, f'{mode}_log.csv')
    df_log = pd.read_csv(log_file)
    
    # Chuyển object_id sang string để map chính xác
    df_log['object_id'] = df_log['object_id'].astype(str)

    # 1. TẠO DICTIONARY CHO METADATA
    # Chúng ta cần cả EBV (để sửa Flux) và Z (để sửa Time & tính Luminosity sau này)
    # Lưu ý: Test set cũng có Z (photometric), dùng tốt!
    meta_map = df_log.set_index('object_id')[['EBV', 'Z']].to_dict('index')

    all_chunks = []
    unique_splits = df_log['split'].unique()

    dtypes = {
        'object_id': str,
        'Time (MJD)': 'float32',
        'Flux': 'float32',
        'Flux_err': 'float32'
    }

    for split_name in tqdm(unique_splits, desc=f"Loading {mode} splits"):
        file_path = os.path.join(BASE_DIR, split_name, f'{mode}_full_lightcurves.csv')
        if not os.path.exists(file_path):
            print(f"Not found {file_path}")
            continue

        df_chunk = pd.read_csv(file_path, dtype=dtypes)

        # 2. MAP METADATA VÀO CHUNK
        # Map EBV và Z vào từng dòng dữ liệu
        # Sử dụng map trực tiếp từ dict sẽ nhanh hơn merge
        # Lưu ý: Cần xử lý trường hợp key không tìm thấy (mặc dù hiếm)
        df_chunk['EBV'] = df_chunk['object_id'].map(lambda x: meta_map.get(x, {}).get('EBV', 0.0))
        df_chunk['Z'] = df_chunk['object_id'].map(lambda x: meta_map.get(x, {}).get('Z', 0.0))
        
        # 3. APPLY DE-EXTINCTION
        # Tính hệ số R_lambda
        df_chunk['R_lambda'] = df_chunk['Filter'].map(LAMBDA)
        
        # Công thức: Flux_corr = Flux * 10^(0.4 * EBV * R)
        correction_factor = 10 ** (0.4 * df_chunk['EBV'] * df_chunk['R_lambda'])
        
        df_chunk['Flux_corrected'] = df_chunk['Flux'] * correction_factor
        df_chunk['Flux_err_corrected'] = df_chunk['Flux_err'] * correction_factor
        
        # Ép kiểu float32 để tiết kiệm RAM
        cols_float = ['Flux_corrected', 'Flux_err_corrected', 'Z', 'EBV']
        for col in cols_float:
            df_chunk[col] = df_chunk[col].astype('float32')

        # 4. CHỌN CỘT CẦN THIẾT
        # Giữ lại Z để dùng cho cell tạo Tensor tiếp theo
        cols_to_keep = [
            'object_id', 
            'Time (MJD)', 
            'Flux_corrected', 
            'Flux_err_corrected', 
            'Filter', 
            'Z' # <--- QUAN TRỌNG
        ]
        all_chunks.append(df_chunk[cols_to_keep])
    
    if not all_chunks:
        print("No data loaded!")
        return pd.DataFrame()
    
    full_df = pd.concat(all_chunks, ignore_index=True)

    if mode == 'train':
        full_df = full_df.merge(df_log[['object_id', 'target']], on='object_id', how='left')
        full_df['target'] = full_df['target'].astype('int8')
    
    return full_df

# --- RUN ---
train_df_clean = process_data(mode='train')
test_df_clean = process_data(mode='test')

# Tạo thư mục nếu chưa có
os.makedirs("/MALLORN-Astronomical-Classification-Challenge/data/processed/", exist_ok=True)

print("Saving to parquet...")
train_df_clean.to_parquet("/MALLORN-Astronomical-Classification-Challenge/data/processed/train_lightcurves_clean.parquet")
test_df_clean.to_parquet("/MALLORN-Astronomical-Classification-Challenge/data/processed/test_lightcurves_clean.parquet")
print("Done!")

In [ ]:
train_df_clean.head()

# DATA PREPROCESSING

In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm

# --- CONFIG ---
INPUT_FILE = '/MALLORN-Astronomical-Classification-Challenge/data/processed/train_lightcurves_clean.parquet'
OUTPUT_FILE = '/MALLORN-Astronomical-Classification-Challenge/data/processed/train_tensor.pt'
MAX_SEQ_LEN = 200
FILTER_MAP = {'u': 0, 'g': 1, 'r': 2, 'i': 3, 'z': 4, 'y': 5, 'Y': 5}

def preprocess(input_path, output_path, is_train=True):
    print(f"Loading data from {input_path}...")
    df = pd.read_parquet(input_path)
    
    # 1. Map Filter ID
    df['filter_id'] = df['Filter'].map(FILTER_MAP).fillna(0).astype('int8')

    # 2. Sort để đảm bảo tính tuần tự thời gian
    df = df.sort_values(by=['object_id', 'Time (MJD)'])

    print("Grouping data...")
    # Thêm 'Z' vào aggregate
    agg_df = df.groupby('object_id').agg({
        'Flux_corrected': list,
        'Flux_err_corrected': list,
        'Time (MJD)': list,
        'filter_id': list,
        'Z': 'first',      # Z là hằng số với mỗi object
        'target': 'first'  # Target (nếu có)
    }).reset_index()
    
    num_samples = len(agg_df)
    
    # --- KHỞI TẠO TENSOR VỚI KÍCH THƯỚC MỚI ---
    # Input Dim = 4: [Flux_Arcsinh, Err_Log, Time_Rest, Redshift]
    X_num = np.zeros((num_samples, MAX_SEQ_LEN, 4), dtype=np.float32) 
    X_cat = np.zeros((num_samples, MAX_SEQ_LEN), dtype=np.int64)
    X_mask = np.zeros((num_samples, MAX_SEQ_LEN), dtype=np.float32)
    y = np.zeros(num_samples, dtype=np.int64) # LongTensor cho CrossEntropy

    ids = agg_df['object_id'].values

    print("Building tensors with Physics transformations...")
    iterator = zip(
        agg_df['Flux_corrected'],
        agg_df['Flux_err_corrected'],
        agg_df['Time (MJD)'],
        agg_df['filter_id'],
        agg_df['Z'],
        agg_df['target']
    )

    for i, (flux, flux_err, times, filters, z, target) in enumerate(tqdm(iterator, total=num_samples)):
        # Convert list -> numpy array
        flux = np.array(flux, dtype=np.float32)
        flux_err = np.array(flux_err, dtype=np.float32)
        times = np.array(times, dtype=np.float32)
        filters = np.array(filters, dtype=np.int64)
        z_val = float(z)

        # --- PHYSICS TRANSFORMATION (QUAN TRỌNG) ---
        
        # 1. Flux Scaling: Dùng Arcsinh thay vì StandardScaler
        # Giữ được tính chất Flux âm của AGN và nén được Flux dương cực lớn của TDE
        flux_trans = np.arcsinh(flux)
        
        # 2. Error Scaling: Logarit
        # Cộng thêm epsilon nhỏ (1e-5) để tránh log(0) nếu có error=0
        err_trans = np.log1p(flux_err)

        # 3. Time Dilation Correction
        # Tính khoảng thời gian trôi qua
        time_delta_obs = times - times[0]
        # Chuyển về Rest-frame (Thời gian thực tại nguồn)
        time_delta_rest = time_delta_obs / (1 + z_val)
        # Normalize: Chia cho 100 ngày để giá trị về khoảng nhỏ (0-1, 0-2...) cho model dễ học
        time_trans = time_delta_rest / 100.0

        seq_len = len(flux)
        limit = min(seq_len, MAX_SEQ_LEN)

        # --- FILL TENSOR ---
        # Feature 0: Flux (Arcsinh)
        X_num[i, :limit, 0] = flux_trans[:limit]
        
        # Feature 1: Error (Log)
        X_num[i, :limit, 1] = err_trans[:limit]
        
        # Feature 2: Time (Rest-frame & Normalized)
        # Giúp model hiểu đúng tốc độ biến thiên của sự kiện
        X_num[i, :limit, 2] = time_trans[:limit]
        
        # Feature 3: Redshift (Z)
        # Cung cấp ngữ cảnh khoảng cách (Distance Context)
        X_num[i, :limit, 3] = z_val

        X_cat[i, :limit] = filters[:limit]
        X_mask[i, :limit] = 1.0 # Mask padding

        # Xử lý target (có thể là NaN trong tập test, ta để tạm 0 hoặc -1)
        if pd.notna(target):
            y[i] = int(target)
        else:
            y[i] = 0

    # Save
    data_dict = {
        'x_num': torch.tensor(X_num),
        'x_cat': torch.tensor(X_cat),
        'x_mask': torch.tensor(X_mask),
        'y': torch.tensor(y),
        'ids': ids
    }
    
    # Không cần lưu scaler nữa vì ta dùng hàm toán học cố định (Arcsinh/Log)
    
    print(f"Saving to {output_path}")
    torch.save(data_dict, output_path)

# --- RUN ---
preprocess(
    input_path=INPUT_FILE,
    output_path=OUTPUT_FILE,
    is_train=True
)

# TRANSFORMER

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

# --- 1. DATASET (Giữ nguyên) ---
class MallornDataset(Dataset):
    def __init__(self, tensors_path, mode='train'):
        data = torch.load(tensors_path, weights_only=False)
        self.x_num = data['x_num'] 
        self.x_cat = data['x_cat'] 
        self.mask = data['x_mask'] 
        self.y = data['y']         
        self.mode = mode
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return {
            'x_num': self.x_num[idx],
            'x_cat': self.x_cat[idx],
            'mask': self.mask[idx],
            'y': self.y[idx]
        }

# --- 2. TIME ENCODING (Giữ nguyên) ---
class TimeEncoding(nn.Module):
    def __init__(self, d_model):
        super(TimeEncoding, self).__init__()
        self.linear = nn.Linear(1, d_model)
        self.activation = nn.Tanh()

    def forward(self, time_values):
        t = time_values.unsqueeze(-1) 
        return self.activation(self.linear(t))

# --- 3. TRANSFORMER MODEL (ĐÃ FIX LỖI DIMENSION) ---
class TDETransformer(nn.Module):
    def __init__(self, 
                 input_dim=4,
                 num_filters=6, 
                 d_model=128, 
                 nhead=4, 
                 num_layers=3, 
                 dim_feedforward=256, 
                 dropout=0.1,
                 num_classes=1):
        super(TDETransformer, self).__init__()
        
        self.content_dim = input_dim - 1 
        self.content_projection = nn.Linear(self.content_dim, d_model)
        
        self.filter_embedding = nn.Embedding(num_filters, d_model)
        self.time_encoding = TimeEncoding(d_model)
        self.input_norm = nn.LayerNorm(d_model)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=nhead, 
            dim_feedforward=dim_feedforward, 
            dropout=dropout,
            batch_first=True,
            norm_first=True 
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.dropout = nn.Dropout(dropout)
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(d_model, 64),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(64, num_classes) 
        )

    def forward(self, x_num, x_cat, mask):
        # 1. Prepare Input
        time_data = x_num[:, :, 2] 
        flux_err = x_num[:, :, :2] 
        z_val = x_num[:, :, 3].unsqueeze(-1)
        content_input = torch.cat([flux_err, z_val], dim=-1) 

        # 2. Embedding
        x_content = self.content_projection(content_input)
        x_filter = self.filter_embedding(x_cat)
        x_time = self.time_encoding(time_data)
        
        x = x_content + x_filter + x_time
        x = self.input_norm(x)
        
        # 3. Transformer
        padding_mask = (mask == 0) 
        x = self.transformer_encoder(x, src_key_padding_mask=padding_mask)
        
        # --- THAY ĐỔI LỚN: MAX POOLING ---
        # Để Max Pooling không bắt phải giá trị padding (thường là 0),
        # ta gán giá trị padding thành âm vô cùng (-1e9)
        mask_expanded = mask.unsqueeze(-1) # [Batch, Seq, 1]
        
        # Fill những chỗ mask=0 bằng số cực nhỏ
        x_masked_for_max = x.masked_fill(mask_expanded == 0, -1e9)
        
        # Lấy giá trị lớn nhất dọc theo chiều Sequence (dim=1)
        # Giúp bắt được đỉnh sáng của TDE bất kể nó nằm ở đâu
        max_embeddings = x_masked_for_max.max(dim=1)[0] # [Batch, d_model]
        
        # 4. Classify
        out = self.dropout(max_embeddings)
        logits = self.classifier(out) 
        
        return logits

# TRAINING LOOP

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import OneCycleLR

# --- CONFIG (Đã cập nhật cho khớp với Model mới) ---
CONFIG = {
    'seed': 42,
    'batch_size': 32,      # Tăng lên 32 để batch norm hoạt động tốt hơn
    'virtual_batch_size': 128,
    'epochs': 15,          # Tăng nhẹ số epoch
    'learning_rate': 2e-4, # LR chuẩn cho Transformer
    'weight_decay': 1e-3,
    'd_model': 128,        # Khớp với Cell 3
    'nhead': 4,
    'num_layers': 3,
    'dim_feedforward': 256,
    'dropout': 0.1,
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'num_workers': 0       # Set = 0 để tránh lỗi Windows/RAM
}

accumulation_steps = CONFIG['virtual_batch_size'] // CONFIG['batch_size']

# --- 1. DATA LOADER (Với Soft Pos Weight) ---
def prepare_loaders(dataset_path):
    print(f"Loading dataset from {dataset_path}...")
    full_dataset = MallornDataset(dataset_path)
    
    # Lấy targets để stratify
    targets = full_dataset.y.numpy()

    train_idx, val_idx = train_test_split(
        np.arange(len(full_dataset)),
        test_size=0.2,
        random_state=CONFIG['seed'],
        stratify=targets
    )

    train_data = Subset(full_dataset, train_idx)
    val_data = Subset(full_dataset, val_idx)

    print(f"Train size: {len(train_data)}, Val size: {len(val_data)}")

    # Loader
    train_loader = DataLoader(
        train_data, 
        batch_size=CONFIG['batch_size'], 
        shuffle=True, 
        drop_last=True,
        num_workers=CONFIG['num_workers'],
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_data, 
        batch_size=CONFIG['batch_size'], 
        shuffle=False,
        num_workers=CONFIG['num_workers'],
        pin_memory=True
    )
    
    # --- SOFT POS_WEIGHT CALCULATION ---
    train_targets = targets[train_idx]
    num_pos = train_targets.sum()
    num_neg = len(train_idx) - num_pos
    
    ratio = num_neg / max(num_pos, 1)
    soft_weight = np.sqrt(ratio) # Căn bậc 2 để giảm bớt sự cực đoan
    
    # Kẹp giá trị trong khoảng hợp lý [3.0, 10.0]
    soft_weight = np.clip(soft_weight, 3.0, 10.0)
    
    print(f"Original Ratio: {ratio:.2f} -> Soft Weight Used: {soft_weight:.2f}")
    
    pos_weight = torch.tensor([soft_weight], dtype=torch.float32).to(CONFIG['device'])

    return train_loader, val_loader, pos_weight

# --- 2. TRAIN FUNCTION (Sửa lỗi thiếu backward) ---
def train_one_epoch(model, loader, criterion, optimizer, device, accumulation_steps, scaler):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    pbar = tqdm(loader, desc='Training', leave=False)

    for i, batch in enumerate(pbar):
        x_num = batch['x_num'].to(device, non_blocking=True)
        x_cat = batch['x_cat'].to(device, non_blocking=True)
        mask = batch['mask'].to(device, non_blocking=True)
        y = batch['y'].float().unsqueeze(1).to(device, non_blocking=True)

        # Robust clamping
        x_num = torch.nan_to_num(x_num, nan=0.0)
        x_num = torch.clamp(x_num, min=-10.0, max=10.0) 

        with torch.amp.autocast('cuda'):
            outputs = model(x_num, x_cat, mask)
            loss = criterion(outputs, y)
            loss = loss / accumulation_steps

        # --- FIX: THÊM BACKWARD ---
        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(loader):
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        loss_val = loss.item() * accumulation_steps
        if not math.isnan(loss_val):
             total_loss += loss_val
        
        pbar.set_postfix({'loss': f"{loss_val:.4f}"})
    
    return total_loss / len(loader)

# --- 3. VALIDATE FUNCTION (Dò tìm Threshold tối ưu) ---
def validate_find_threshold(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    
    # Dùng list để gom batch array nhanh hơn extend từng phần tử
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for batch in tqdm(loader, desc='Validating', leave=False):
            x_num = batch['x_num'].to(device, non_blocking=True)
            x_cat = batch['x_cat'].to(device, non_blocking=True)
            mask = batch['mask'].to(device, non_blocking=True)
            y = batch['y'].float().unsqueeze(1).to(device, non_blocking=True)

            x_num = torch.nan_to_num(x_num, nan=0.0)
            x_num = torch.clamp(x_num, min=-10.0, max=10.0)

            outputs = model(x_num, x_cat, mask)
            
            loss = criterion(outputs, y)
            total_loss += loss.item()

            probs = torch.sigmoid(outputs)
            all_preds.append(probs.cpu().numpy())
            all_targets.append(y.cpu().numpy())

    # Nối mảng lớn một lần
    if not all_preds: return float('inf'), 0, 0, 0, 0.5
    
    all_probs = np.concatenate(all_preds).ravel()
    all_targets = np.concatenate(all_targets).ravel()

    # --- THRESHOLD SEARCH ---
    best_f1 = 0
    best_thresh = 0.001
    prec, rec = 0, 0
    
    # Quét từ 0.1 đến 0.9
    for t in np.arange(0.1, 0.9, 0.05):
        preds = (all_probs > t).astype(int)
        if preds.sum() == 0: continue
        
        score = f1_score(all_targets, preds)
        if score > best_f1:
            best_f1 = score
            best_thresh = t
            prec = precision_score(all_targets, preds, zero_division=0)
            rec = recall_score(all_targets, preds)

    avg_loss = total_loss / len(loader)
    
    # Debug: In ra xác suất trung bình để biết model tự tin cỡ nào
    print(f"   [Debug] Avg Prob: {all_probs.mean():.4f} | Max Prob: {all_probs.max():.4f}")
    
    return avg_loss, best_f1, prec, rec, best_thresh

# --- 4. MAIN RUN ---
def run_training_final():
    # Load Data
    train_loader, val_loader, _ = prepare_loaders('/MALLORN-Astronomical-Classification-Challenge/data/processed/train_tensor.pt')
    
    # --- THAY ĐỔI QUYẾT ĐỊNH ---
    # Gán cứng pos_weight = 2.0 (Thay vì để code tự tính ra 5 hay 10)
    # Ý nghĩa: Chỉ ưu tiên TDE gấp đôi nhiễu thôi, không được spam.
    print("!!! FORCE OVERRIDE: Setting pos_weight = 5.0 !!!")
    pos_weight = torch.tensor([5.0]).to(CONFIG['device'])
    
    # Init Model (input_dim=4)
    model = TDETransformer(
        input_dim=4, 
        d_model=CONFIG['d_model'],
        nhead=CONFIG['nhead'],
        num_layers=CONFIG['num_layers'],
        dropout=CONFIG['dropout']
    ).to(CONFIG['device'])
    
    # Loss & Optimizer
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
    
    scheduler = OneCycleLR(optimizer, max_lr=CONFIG['learning_rate'], 
                          steps_per_epoch=len(train_loader), epochs=CONFIG['epochs'])
    
    scaler = torch.amp.GradScaler('cuda')
    
    best_f1_global = 0.0
    print("\n--- START TRAINING (Low Weight Strategy) ---")
    
    for epoch in range(CONFIG['epochs']):
        # Train
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, CONFIG['device'], accumulation_steps, scaler)
        
        # Validate
        val_loss, val_f1, val_prec, val_rec, thresh = validate_find_threshold(model, val_loader, criterion, CONFIG['device'])
        
        scheduler.step()
        
        print(f"Epoch {epoch+1:02d} | Train: {train_loss:.4f} | Val: {val_loss:.4f} | "
              f"F1: {val_f1:.4f} (at {thresh:.2f}) [P: {val_prec:.2f}, R: {val_rec:.2f}]")
        
        if val_f1 > best_f1_global:
            best_f1_global = val_f1
            torch.save(model.state_dict(), '/MALLORN-Astronomical-Classification-Challenge/models/best_physics_model.pth')
            print("--> Model Saved!")

# --- EXECUTE ---
if __name__ == '__main__':
    run_training_final()

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

def run_inference():
    print("\n--- START INFERENCE ---")
    
    # 1. Load Test Data
    # Đảm bảo bạn đã chạy preprocess cho test set để tạo ra file này
    test_file = '/MALLORN-Astronomical-Classification-Challenge/data/processed/test_tensor.pt'
    if not os.path.exists(test_file):
        # Nếu chưa có thì chạy preprocess cho test
        print("Generating test tensors...")
        preprocess(
            input_path='/MALLORN-Astronomical-Classification-Challenge/data/processed/test_lightcurves_clean.parquet',
            output_path=test_file,
            is_train=False
        )
        
    test_dataset = MallornDataset(test_file, mode='test')
    test_loader = DataLoader(
        test_dataset, 
        batch_size=CONFIG['batch_size'] * 2, # Batch lớn hơn cho nhanh (không cần backward)
        shuffle=False, 
        num_workers=0,
        pin_memory=True
    )
    
    # 2. Load Best Model
    model = TDETransformer(
        input_dim=4, # Khớp với config train
        d_model=CONFIG['d_model'],
        nhead=CONFIG['nhead'],
        num_layers=CONFIG['num_layers'],
        dropout=0.0 # Không dropout khi dự đoán
    ).to(CONFIG['device'])
    
    model_path = '/MALLORN-Astronomical-Classification-Challenge/models/best_physics_model.pth'
    if not os.path.exists(model_path):
        print(f"Error: Model not found at {model_path}. Did you train successfully?")
        return

    print(f"Loading model from {model_path}")
    model.load_state_dict(torch.load(model_path, map_location=CONFIG['device']))
    model.eval()
    
    # 3. Predict
    all_preds = []
    all_ids = test_dataset.x_num # ID nằm trong data gốc lúc save, nhưng dataset class chưa trả về ID
    # Fix lại cách lấy ID từ file saved
    data = torch.load(test_file, weights_only=False)
    object_ids = data['ids']
    
    predictions = {}
    
    idx_counter = 0
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Predicting"):
            x_num = batch['x_num'].to(CONFIG['device'])
            x_cat = batch['x_cat'].to(CONFIG['device'])
            mask = batch['mask'].to(CONFIG['device'])
            
            x_num = torch.nan_to_num(x_num, nan=0.0)
            x_num = torch.clamp(x_num, min=-10.0, max=10.0)
            
            logits = model(x_num, x_cat, mask)
            probs = torch.sigmoid(logits).cpu().numpy().flatten()
            
            # Map predictions to Object IDs
            batch_size = len(probs)
            current_ids = object_ids[idx_counter : idx_counter + batch_size]
            
            for obj_id, prob in zip(current_ids, probs):
                predictions[obj_id] = prob
                
            idx_counter += batch_size

    # 4. Create Submission DataFrame
    # Load sample submission để đảm bảo đúng format (nếu có)
    # Hoặc tạo mới từ predictions
    sub_df = pd.DataFrame({
        'object_id': list(predictions.keys()),
        'target': list(predictions.values())
    })
    
    # Quan trọng: Dùng Threshold tối ưu đã tìm được lúc Train
    # Nếu bạn nhớ threshold (ví dụ 0.25), bạn có thể convert sang 0/1 ở đây
    # Nhưng thường Kaggle yêu cầu xác suất (float). Nếu yêu cầu nhãn 0/1:
    # sub_df['target'] = (sub_df['target'] > BEST_THRESHOLD).astype(int)
    
    # Lưu file
    output_csv = 'submission.csv'
    sub_df.to_csv(output_csv, index=False)
    print(f"Submission saved to {output_csv}. Rows: {len(sub_df)}")
    print(sub_df.head())

if __name__ == '__main__':
    run_inference()